In [1]:
# Script from steps 1-7
import mne
import pandas as pd
import os
import numpy as np
import csv

In [21]:
# bdf_file = 'subj03_session2'

# bdf_file_path = os.path.join('eeg_data', 'bdf', f"{bdf_file}.bdf") 

# # Read the BDF file
# raw = mne.io.read_raw_bdf(bdf_file_path, preload=True)
fiff_file = 'subj03_session1'
fiff_file_path = 'subj03_session1.fif'
raw = mne.io.read_raw_fif(fiff_file, preload=True)
print("Channels:", raw.ch_names)

Opening raw data file subj03_session1...


/var/folders/v0/v0sq4xs162l4_3rly20g6yv00000gq/T/ipykernel_28571/19557071.py:9: RuntimeWarning: This filename (subj03_session1) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fiff_file, preload=True)


FileNotFoundError: fname does not exist: "/Users/qianh1/Code/BCI/alljoined_preprocessing/subj03_session1"

In [22]:
events = mne.find_events(raw, stim_channel='Status', shortest_event=1)
print("events.shape:", events.shape)
print("events[:10]:", events[:10])
print("events[-10:]:", events[-10:])

Trigger channel Status has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
11810 events found on stim channel Status
Event IDs: [    1     2     3     4     5     6     7     8     9    10    11    12
    13    14    15    16    17    18    19    20    21    22    23    24
    25    26    27    28    29    30    31    32    33    34    35    36
    37    38    39    40    41    42    43    44    45    46    47    48
    49    50    51    52    53    54    55    56    57    58    59    60
    61    62    63    64    65    66    67    68    69    70    71    72
    73    74    75    76    77    78    79    80    81    82    83    84
    85    86    87    88    89    90    91    92    93    94    95    96
    97    98    99   100   101   102   103   104   105   106   107   108
   109   110   111   112   113   114   115   116   117   118   119   120
   121   122   123   124   125   126   127   128   129   130   131   132
   133   134   1

In [23]:
unique_event_ids = np.unique(events[:, 2])  # Extract unique event IDs
print(unique_event_ids)

'''
1. **198:** start
2. **199:** stop
3. **201:** train block
4. **202-217:** testing blocks
5. **START_RECORD = 198; % the trigger to start the EEG recording**
6. **STOP_RECORD = 199; % the trigger to stop the EEG recording**
7. **CORRECT_HIT = 254; % the trigger that records a correct hit to a catch stimulus**
8. **FALSE_ALARM = 253; % the trigger that records a false hit to an experimental stimulus**
9. **CORRECT_REJ = 252; % the trigger that records a correct no-key to an experiment stimulus**
10. **MISS = 251; % the trigger that records a false no-key to a catch stimulus**
'''

[    1     2     3     4     5     6     7     8     9    10    11    12
    13    14    15    16    17    18    19    20    21    22    23    24
    25    26    27    28    29    30    31    32    33    34    35    36
    37    38    39    40    41    42    43    44    45    46    47    48
    49    50    51    52    53    54    55    56    57    58    59    60
    61    62    63    64    65    66    67    68    69    70    71    72
    73    74    75    76    77    78    79    80    81    82    83    84
    85    86    87    88    89    90    91    92    93    94    95    96
    97    98    99   100   101   102   103   104   105   106   107   108
   109   110   111   112   113   114   115   116   117   118   119   120
   121   122   123   124   125   126   127   128   129   130   131   132
   133   134   135   136   137   138   139   140   141   142   144   145
   146   147   148   149   150   151   152   153   154   155   156   157
   158   159   160   161   162   163   164   165   

'\n1. **198:** start\n2. **199:** stop\n3. **201:** train block\n4. **202-217:** testing blocks\n5. **START_RECORD = 198; % the trigger to start the EEG recording**\n6. **STOP_RECORD = 199; % the trigger to stop the EEG recording**\n7. **CORRECT_HIT = 254; % the trigger that records a correct hit to a catch stimulus**\n8. **FALSE_ALARM = 253; % the trigger that records a false hit to an experimental stimulus**\n9. **CORRECT_REJ = 252; % the trigger that records a correct no-key to an experiment stimulus**\n10. **MISS = 251; % the trigger that records a false no-key to a catch stimulus**\n'

In [25]:
# Output our events into a .csv file
# csv_file_path = os.path.join('eeg_data', 'raw_csv', f"{fiff_file}.csv") 
csv_file_path = 'raw_subj3_session1.csv'


# os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)

with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header row
    writer.writerow(['onset', 'sig', 'code'])
    
    # Write the data rows
    for event in events:
        writer.writerow(event)

the cell below consolidates the triplet event codes in the raw csv file in a single event code in the parsed csv file. Oddballs are ignored. If an error is outputted, the raw csv file likely has a phantom event code, and will require manual inspection to remove it.

In [35]:
def create_new_row(row, img_idx):
    new_row = row.copy()
    new_row.iloc[2] = img_idx

    return new_row

raw_eeg_df = pd.read_csv(csv_file_path)
new_df = pd.DataFrame(columns=raw_eeg_df.columns)

i = 2
block_number = 1
block_trigger_offset = 150
error_count = 0

while i < raw_eeg_df.shape[0] - 2:
    
    # New block, increment block number and add a block code to new csv
    if raw_eeg_df.iloc[i, 2] == 65536:
        block_number += 1
        i+=3
        
    first_row_event_code = raw_eeg_df.iloc[i, 2]
    second_row_event_code = raw_eeg_df.iloc[i+1, 2]
    third_row_event_code = raw_eeg_df.iloc[i+2, 2]

   #  We use this for the old trigger script, for sessions 4 and lower.
    if first_row_event_code > 100:
        second_row_event_code -= 100
        first_row_event_code += 50

    # 254 - Oddball - Correct - hit 
    if third_row_event_code == 254 and first_row_event_code == block_number+block_trigger_offset:
        pass 
    
    # # 251 - Oddball - Miss - no hit
    elif third_row_event_code == 251 and first_row_event_code == block_number+block_trigger_offset:
        pass
        
    # 252 - Correct - no hit  
    elif third_row_event_code == 252 and first_row_event_code == block_number:
        img_idx = ((block_number - 1) % 8) * 120 + second_row_event_code
        modified_row = create_new_row(raw_eeg_df.iloc[i], img_idx)
        new_df = new_df._append(modified_row, ignore_index=True)
        
    # 253 - False hit 
    elif third_row_event_code == 253 and first_row_event_code == block_number:
        img_idx = ((block_number - 1) % 8) * 120 + second_row_event_code
        modified_row = create_new_row(raw_eeg_df.iloc[i], img_idx)
        new_df = new_df._append(modified_row, ignore_index=True)

    else:
        if error_count < 10:
            error_count += 1
            print(f'Error in line {i+2}:', first_row_event_code, third_row_event_code)

    # Jump 3 rows for next trial
    i += 3

# Save data
# final_output_path = csv_file_path.replace('raw', 'parsed')
# os.makedirs(os.path.dirname(final_output_path), exist_ok=True)
final_output_path = 'parsed_subj3_session1.csv'
new_df.to_csv(final_output_path, index=False)
print('Saved to: ', final_output_path)

Saved to:  parsed_subj3_session1.csv


In [7]:
# eeg_data/parsed_csv/subj04_session1.csv
# final_output_path="eeg_data/parsed_csv/subj10_session2.csv"

In [37]:
# Confirm that all images appear 4 times
# Step 1: Load the CSV file

events_df = pd.read_csv(final_output_path)


code_counts = events_df.groupby('code').size()
codes_less = code_counts[code_counts < 4].index.tolist()
codes_more = code_counts[code_counts > 4].index.tolist()

print("Stimuli that appear less than 4 times:", codes_less)
print("Stimuli that appear more than 4 times:", codes_more)

Stimuli that appear less than 4 times: [2, 4, 6, 8, 11, 12, 16, 17, 21, 22, 23, 26, 27, 29, 32, 34, 35, 36, 38, 39, 42, 45, 46, 49, 55, 56, 60, 63, 66, 78, 90, 97, 98, 99, 100, 101, 103, 119, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 924]
Stimuli that appear more than 4 times: []


In [38]:
parsed_eeg_df = pd.read_csv(final_output_path)

# Count the number of trials
num_trials = parsed_eeg_df.shape[0]
print("Number of trials:", num_trials)

# Count the number of repetitions for each unique image
image_counts = parsed_eeg_df['code'].value_counts()
print("Number of repetitions for unique images:")
print(image_counts.value_counts())

Number of trials: 3561
Number of repetitions for unique images:
count
4    801
2    120
3     39
Name: count, dtype: int64


In [40]:
# Prepare the events array required by MNE
events_array = events_df[['onset', 'sig', 'code']].to_numpy()

# Step 2: Load the BDF file
# raw = mne.io.read_raw_bdf(fiff_file, preload=True)
raw = mne.io.read_raw_fif(fiff_file_path, preload=True)


# Step 3 & 4: Override events
# This example assumes you want to replace all existing events
raw.add_events(events_array, stim_channel='Status', replace=True)

# Step 5: Save the modified BDF file
# Note: MNE does not support saving in BDF format directly, so consider saving in a different format or use a different library for BDF writing
fif_path = os.path.join('eeg_data', 'fif', f"{fiff_file}_eeg.fif") 
os.makedirs(os.path.dirname(fif_path), exist_ok=True)
raw.save(fif_path, overwrite=True)


# Step 6: Check the new BDF file
raw_fif = mne.io.read_raw_fif(fif_path)
events = mne.find_events(raw_fif)
print("events.shape:", events.shape)

Opening raw data file subj03_session1.fif...


/var/folders/v0/v0sq4xs162l4_3rly20g6yv00000gq/T/ipykernel_28571/3541505333.py:6: RuntimeWarning: This filename (subj03_session1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fiff_file_path, preload=True)


Isotrak not found
    Range : 0 ... 1664511 =      0.000 ...  3250.998 secs
Ready.
Reading 0 ... 1664511  =      0.000 ...  3250.998 secs...
Writing /Users/qianh1/Code/BCI/alljoined_preprocessing/eeg_data/fif/subj03_session1_eeg.fif
Closing /Users/qianh1/Code/BCI/alljoined_preprocessing/eeg_data/fif/subj03_session1_eeg.fif
[done]
Opening raw data file eeg_data/fif/subj03_session1_eeg.fif...
Isotrak not found
    Range : 0 ... 1664511 =      0.000 ...  3250.998 secs
Ready.
3561 events found on stim channel Status
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 